In [6]:
import os
import shutil

import tensorflow as tf
import pandas as pd

import json
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tf.get_logger().setLevel('ERROR')

In [7]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VICTUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [167]:
df_old = pd.read_csv("Resume.csv")
df_old = df_old[['Resume_str','Category']]
df_old.columns = ['resume','category']
df_new = pd.read_csv("UpdatedResumeDataSet.csv")
df_new.columns = ['category','resume']
df_new.head()

,category,resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [168]:
import re
def clean_resume(resume_str, stop_words):
    resume_str   = resume_str.apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
    cleaned_resume = resume_str.str.strip()
    cleaned_resume = cleaned_resume.str.replace('\s+', ' ', regex=True)
    cleaned_resume = cleaned_resume.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
    return cleaned_resume

In [169]:
df = pd.concat([df_old,df_new])
df['resume'] = clean_resume(df.resume,stop)
df

,resume,category
0,HR ADMINISTRATORMARKETING ASSOCIATE HR ADMINIS...,HR
1,HR SPECIALIST US HR OPERATIONS Summary Versati...,HR
2,HR DIRECTOR Summary Over 20 years experience r...,HR
3,HR SPECIALIST Summary Dedicated Driven Dynamic...,HR
4,HR MANAGER Skill Highlights HR SKILLS HR Depar...,HR
...,...,...
957,Computer Skills Proficient MS office Word Basi...,Testing
958,Willingness accept challenges Positive thinkin...,Testing
959,PERSONAL SKILLS Quick learner Eagerness learn ...,Testing
960,COMPUTER SKILLS SOFTWARE KNOWLEDGE MSPower Poi...,Testing


In [170]:
class_names = df.category.unique()
#class mapping
class_mapping = {label:idx for idx,label in enumerate(class_names)}

df.category = df.category.map(class_mapping)

In [64]:
from sklearn.model_selection import train_test_split
X = df.resume
y = df.category
X_train,X_valid,y_train,y_valid = train_test_split(X,y,stratify = y,test_size = 0.2,random_state= 42)

In [65]:
#tokenizer 
vocab_size = 10000
oov_token = "<OOV>"
tokenizer = Tokenizer(num_words = vocab_size,oov_token = oov_token)
tokenizer.fit_on_texts(X_train)

training_sequences = tokenizer.texts_to_sequences(X_train)
training_padded = pad_sequences(training_sequences,padding = 'post',maxlen = 768)

validation_sequences = tokenizer.texts_to_sequences(X_valid)
validation_padded = pad_sequences(validation_sequences,padding = 'post',maxlen = 768)

In [155]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64, input_length=768),
    tf.keras.layers.Conv1D(32, 5, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    #dropout
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(len(class_mapping), activation='softmax')
])

model.compile(loss = 'sparse_categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
model.summary()

Model: "sequential_55"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_57 (Embedding)    (None, 768, 64)           640000    
                                                                 
 conv1d_33 (Conv1D)          (None, 764, 32)           10272     
                                                                 
 global_average_pooling1d_44  (None, 32)               0         
  (GlobalAveragePooling1D)                                       


                                                                 
 dropout_35 (Dropout)        (None, 32)                0         
                                                                 
 dense_87 (Dense)            (None, 48)                1584      
                                                                 
Total params: 651,856
Trainable params: 651,856
Non-trainable params: 0
_________________________________________________________________


In [156]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',patience = 5,restore_best_weights = True)

num_epochs = 1000
history = model.fit(training_padded,y_train,epochs = num_epochs,validation_data = (validation_padded,y_valid),callbacks = [early_stopping])

Epoch 1/1000


87/87 [==============================] - 1s 11ms/step - loss: 3.8052 - accuracy: 0.0414 - val_loss: 3.6100 - val_accuracy: 0.0551
Epoch 2/1000
87/87 [==============================] - 1s 7ms/step - loss: 3.4992 - accuracy: 0.0588 - val_loss: 3.3499 - val_accuracy: 0.0986
Epoch 3/1000
87/87 [==============================] - 1s 7ms/step - loss: 3.2933 - accuracy: 0.0929 - val_loss: 3.1624 - val_accuracy: 0.1391
Epoch 4/1000
87/87 [==============================] - 1s 8ms/step - loss: 3.1437 - accuracy: 0.1179 - val_loss: 2.9938 - val_accuracy: 0.2072
Epoch 5/1000
87/87 [==============================] - 1s 8ms/step - loss: 2.9683 - accuracy: 0.1655 - val_loss: 2.8259 - val_accuracy: 0.2478
Epoch 6/1000
87/87 [==============================] - 1s 7ms/step - loss: 2.7893 - accuracy: 0.2061 - val_loss: 2.6577 - val_accuracy: 0.3174
Epoch 7/1000
87/87 [==============================] - 1s 8ms/step - loss: 2.6436 - accuracy: 0.2540 - val_loss: 2.5088 - val_accuracy: 0.3841
Epoch 8/1000
87/87

In [157]:
model.evaluate(validation_padded,y_valid)

22/22 [==============================] - 0s 13ms/step - loss: 1.1333 - accuracy: 0.7188


[1.1333305835723877, 0.7188405990600586]

In [161]:
#save tokenizer and model
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
model.save("resume_model.h5")


In [172]:
class_mapping

{'HR': 0,
 'DESIGNER': 1,
 'INFORMATION-TECHNOLOGY': 2,
 'TEACHER': 3,
 'ADVOCATE': 4,
 'BUSINESS-DEVELOPMENT': 5,
 'HEALTHCARE': 6,
 'FITNESS': 7,
 'AGRICULTURE': 8,
 'BPO': 9,
 'SALES': 10,
 'CONSULTANT': 11,
 'DIGITAL-MEDIA': 12,
 'AUTOMOBILE': 13,
 'CHEF': 14,
 'FINANCE': 15,
 'APPAREL': 16,
 'ENGINEERING': 17,
 'ACCOUNTANT': 18,
 'CONSTRUCTION': 19,
 'PUBLIC-RELATIONS': 20,
 'BANKING': 21,
 'ARTS': 22,
 'AVIATION': 23,
 'Data Science': 24,
 'Advocate': 25,
 'Arts': 26,
 'Web Designing': 27,
 'Mechanical Engineer': 28,
 'Sales': 29,
 'Health and fitness': 30,
 'Civil Engineer': 31,
 'Java Developer': 32,
 'Business Analyst': 33,
 'SAP Developer': 34,
 'Automation Testing': 35,
 'Electrical Engineering': 36,
 'Operations Manager': 37,
 'Python Developer': 38,
 'DevOps Engineer': 39,
 'Network Security Engineer': 40,
 'PMO': 41,
 'Database': 42,
 'Hadoop': 43,
 'ETL Developer': 44,
 'DotNet Developer': 45,
 'Blockchain': 46,
 'Testing': 47}